# Задание 3.1 - Сверточные нейронные сети (Convolutional Neural Networks)

Это последнее задание на numpy, вы до него дожили! Остался последний марш-бросок, дальше только PyTorch.

В этом задании вы реализуете свою собственную сверточную нейронную сеть.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_layer_gradient, check_layer_param_gradient, check_model_gradient
from layers import FullyConnectedLayer, ReLULayer, ConvolutionalLayer, MaxPoolingLayer, Flattener
from model import ConvNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD, Adam
from metrics import multiclass_accuracy


In [3]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train[0].shape

(28, 28)

# Загружаем данные

На этот раз мы не будем их преобразовывать в один вектор, а оставим размерности (num_samples, 32, 32, 3).

In [5]:
from sklearn.model_selection import train_test_split
def prepare_for_neural_network(train_X, test_X):
    train_X = train_X.astype(np.float) / 255.0
    test_X = test_X.astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_X, axis = 0)
    train_X -= mean_image
    test_X -= mean_image
    return train_X, test_X
    
train_X, test_X = prepare_for_neural_network(x_train, x_test)
# Split train into train and val
train_X, val_X, train_y, val_y = train_test_split(train_X, y_train, random_state=42, test_size=0.3)
test_y = y_test

In [6]:
train_X = train_X.reshape((-1, 28, 28, 1))
val_X = val_X.reshape((-1, 28, 28, 1))
test_X = test_X.reshape((-1, 28, 28, 1))

In [7]:
train_X.shape, val_X.shape, test_X.shape

((42000, 28, 28, 1), (18000, 28, 28, 1), (10000, 28, 28, 1))

# Оптимизатор и код для тренировки 
Должен заработать с кодом из прошлого задания без изменений!

In [8]:
# model = ConvNet(input_shape=(28,28,1), n_output_classes=10, conv1_channels=2, conv2_channels=2)
# dataset = Dataset(train_X[:16], train_y[:16], val_X[:16], val_y[:16])
# trainer = Trainer(model, dataset, SGD(), batch_size=16, learning_rate=1e-4)

# loss_history, train_history, val_history = trainer.fit()

In [9]:
# plt.plot(train_history)
# plt.plot(val_history)

## Итак, оверфитим маленький набор данных


In [10]:
# bottle_resized = resize(bottle, (140, 54))

# def resize(arr, shape):
#     # img = cv2.imread('your_image.jpg')
#     res = cv2.resize(arr, dsize=shape, interpolation=cv2.INTER_CUBIC)
#     return res
    
# train_X_resized = train_X.repeat(8, axis=1).repeat(8, axis=2)
# val_X_resized = val_X.repeat(8, axis=1).repeat(8, axis=2)
# train_X_resized.shape

In [11]:
train_X_resized = train_X[:128].repeat(8, axis=1).repeat(8, axis=2)
val_X_resized = val_X[:128].repeat(8, axis=1).repeat(8, axis=2)
train_X_resized.shape

(128, 224, 224, 1)

In [12]:
data_size = 128
model = ConvNet(input_shape=(224,224,1), n_output_classes=10, conv1_channels=40, conv2_channels=40)
dataset = Dataset(train_X_resized[:data_size], train_y[:data_size], val_X_resized[:data_size], val_y[:data_size])
# TODO: Change any hyperparamers or optimizators to reach 1.0 training accuracy in 50 epochs or less
# Hint: If you have hard time finding the right parameters manually, try grid search or random search!
trainer = Trainer(model, dataset, Adam(), learning_rate=1e-3, num_epochs=50, batch_size=64)

loss_history, train_history, val_history = trainer.fit()

Loss: 6.044286, Train accuracy: 0.101562, val accuracy: 0.093750
Loss: 6.583954, Train accuracy: 0.132812, val accuracy: 0.101562
Loss: 6.044286, Train accuracy: 0.140625, val accuracy: 0.093750
Loss: 5.593975, Train accuracy: 0.117188, val accuracy: 0.085938


KeyboardInterrupt: 

In [ ]:
plt.plot(train_history)
plt.plot(val_history)